<a href="https://colab.research.google.com/github/carloscastillo10/predicting-academic-performance/blob/main/annex-b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CASO 1

El **caso 1** debe contener la siguiente estructura:

| fake_identificacion | nombre_materia_1 | num_repr_materia_1 | nombre_materia_2 | num_repr_materia_2 | nombre_materia_n | num_repr_materia_n |
| ------------------- | ---------------- | ---------------- | ---------------- | ---------------- | ---------------- | ---------------- |
| MWGG30052559625869  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n |
| KWUQ54712342267224  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n | nota_final_materia_1 | nota_final_materia_1 | nota_final_materia_n |
| ZZNW63915190724931  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n |
| YPKN05597705348880  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n |
| WHAX47312762710440  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n | nota_final_materia_1 | nota_final_materia_1 | nota_final_materia_n |
| CDKP20450612579481  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n |
| TREG36200105332019  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n |
| LQOB86905081708835  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n | nota_final_materia_1 | nota_final_materia_1 | nota_final_materia_n |
| ZISM01074058271119  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n |
| XNCB51659919828958  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n | nota_final_materia_1 | nota_final_materia_1 | nota_final_materia_n |
| NCHA16213110074700  | nota_final_materia_1 | num_reprl_materia_1 | nota_final_materia_2 | num_reprl_materia_2 | nota_final_materia_n | num_reprl_materia_n |
 
**Consideraciones**:
* Se tomaron en cuenta solo los estudiantes que si tienen al menos una nota registrada.
* Se colocaron a nivel de columnas las 57 materias que contiene la malla curricular de la carrera de TI.
* Se extrajo solo la nota registrada en la columna **`final_nota`** de cada materia
* Si la materia se tomó en varios ciclos, se colocó la nota del periodo mas reciente.
* Si el estudiante aun no ha tomado alguna de las materias se deja un valor en blanco para esa materia.
* Se calculo el numero de reprobaciones por cada materia, si no ha reprobado en ninguna ocasión la materia se coloca el valor de 0.
* Se colocó los datos de matriculas del periodo mas reciente de cada estudiante


In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
pd.options.display.max_columns = 200
pd.set_option('mode.chained_assignment', None)

In [ ]:
def load_data(path):
    data = pd.read_csv(path, low_memory=False)
    data.columns = data.columns.str.lower()
    return data

## Leer datos de matrículas y de notas

### Datos de matrículas

In [ ]:
enrollment_path = 'result/completed-enrollment-data.csv'
enrollment_data = load_data(enrollment_path)

### Datos de notas

In [ ]:
grades_path = 'result/completed-grades-data.csv'
grades_data = load_data(grades_path)

## Integrar los datos

In [ ]:
def set_grades_columns():
    subjects = []
    for subject in grades_data.curso_nombre.unique():
        subjects.extend([subject, f'num_reprob_{subject}'])

    columns_grades = ['fake_identificacion'] + subjects
    
    return columns_grades

def get_enrollment_data_by_recent_period(fake_identification):
    result = enrollment_data[
        enrollment_data.fake_identificacion == fake_identification
    ].sort_values(by = 'codigo_periodo', ascending = False)
    
    if not result.empty:
        return result.iloc[[0]]

def get_student_subjects(fake_identification):
    return grades_data[
        grades_data.fake_identificacion == fake_identification
    ]['curso_nombre'].unique()


def get_final_score_by_subject(subject_data): 
    final_score = np.NAN

    if not subject_data.empty:
        final_score = subject_data.iloc[[0]]['final_nota'].to_list()[0]
    
    return final_score

def get_total_failures_by_subject(subject_data):
    total_failures = len(subject_data[subject_data.status == 'REPROBADA'])
    return total_failures

def get_grades_data_by_recent_period_and_subject(fake_identification, subject_name):
    subject_data = grades_data[
        (grades_data.fake_identificacion == fake_identification)
        & (grades_data.curso_nombre == subject_name)
    ].sort_values(by = 'periodo_academico', ascending = False)
    
    final_score = get_final_score_by_subject(subject_data)
    total_failures = get_total_failures_by_subject(subject_data)
    
    return final_score, total_failures

### Analisis individual de un estudiante

In [ ]:
fake_identification = 'MCNO16001921933764'
student_enrollment_data = get_enrollment_data_by_recent_period(fake_identification)

In [ ]:
subjects = list(grades_data.curso_nombre.unique())
columns_grades = set_grades_columns()
student_grades_data = pd.DataFrame(columns=columns_grades)

In [ ]:
student_grades_data = pd.concat([student_grades_data, pd.DataFrame({'fake_identificacion': [fake_identification]})])
student_subjects = get_student_subjects(fake_identification)

for subject_name in student_subjects:
    subject_final_score, subject_total_failures  = get_grades_data_by_recent_period_and_subject(fake_identification, subject_name)
    student_grades_data[subject_name] = subject_final_score
    student_grades_data[f'num_reprob_{subject_name}'] = subject_total_failures
student_grades_data

,fake_identificacion,COMUNICACION DE DATOS,num_reprob_COMUNICACION DE DATOS,FUNDAMENTOS DE HARDWARE,num_reprob_FUNDAMENTOS DE HARDWARE,ESTRUCTURAS DISCRETAS,num_reprob_ESTRUCTURAS DISCRETAS,ESTRUCTURAS DE DATOS,num_reprob_ESTRUCTURAS DE DATOS,COMUNICACION TECNICA Y PROFESI,num_reprob_COMUNICACION TECNICA Y PROFESI,FUNDAMENTOS DE INTERACCION HUM,num_reprob_FUNDAMENTOS DE INTERACCION HUM,METODOLOGIA DE LA INVEST Y TEC,num_reprob_METODOLOGIA DE LA INVEST Y TEC,COMPUTACION Y SOCIEDAD,num_reprob_COMPUTACION Y SOCIEDAD,FUNDAMENTOS DE TECNOLOGIAS,num_reprob_FUNDAMENTOS DE TECNOLOGIAS,ARQUITECTURA DE COMPUTADORAS,num_reprob_ARQUITECTURA DE COMPUTADORAS,MODELADO DE SISTEMAS,num_reprob_MODELADO DE SISTEMAS,PRACTICUM II,num_reprob_PRACTICUM II,ESTADISTICA PARA LAS ING Y ARQ,num_reprob_ESTADISTICA PARA LAS ING Y ARQ,CALCULO DIFERENCIAL,num_reprob_CALCULO DIFERENCIAL,FUNDAMENTOS DE INGENIERIA DE S,num_reprob_FUNDAMENTOS DE INGENIERIA DE S,FUNDAMENTOS DE BASE DE DATOS,num_reprob_FUNDAMENTOS DE BASE DE DATOS,ADMINISTRACION DE BASE DE DATO,num_reprob_ADMINISTRACION DE BASE DE DATO,ALGORITMOS Y RESOLUCION DE PRO,num_reprob_ALGORITMOS Y RESOLUCION DE PRO,COMPUTACION UBICUA,num_reprob_COMPUTACION UBICUA,FUNDAMENTOS DE PROGRAMACION,num_reprob_FUNDAMENTOS DE PROGRAMACION,PROGRAMACION ORIENTADA A OBJET,num_reprob_PROGRAMACION ORIENTADA A OBJET,TECNOLOGIAS WEB,num_reprob_TECNOLOGIAS WEB,DESARROLLO WEB,num_reprob_DESARROLLO WEB,ETICA Y MORAL,num_reprob_ETICA Y MORAL,"HUMANISMO, UNIVERSIDAD Y CULTU","num_reprob_HUMANISMO, UNIVERSIDAD Y CULTU",ANTROPOLOGIA BASICA,num_reprob_ANTROPOLOGIA BASICA,FUNDAMENTOS MATEMATICOS,num_reprob_FUNDAMENTOS MATEMATICOS,ALGEBRA LINEAL,num_reprob_ALGEBRA LINEAL,CALCULO INTEGRAL,num_reprob_CALCULO INTEGRAL,REDES DE DISPOSITIVOS,num_reprob_REDES DE DISPOSITIVOS,ARQUITECTURA DE REDES,num_reprob_ARQUITECTURA DE REDES,PLATAFORMAS EMERGENTES,num_reprob_PLATAFORMAS EMERGENTES,GESTION DE PROYECTOS,num_reprob_GESTION DE PROYECTOS,PRACTICUM I,num_reprob_PRACTICUM I,PLANIFICACION ESTRATEGICA Y SI,num_reprob_PLANIFICACION ESTRATEGICA Y SI,METODOLOGIAS DE DESARROLLO,num_reprob_METODOLOGIAS DE DESARROLLO,GESTION DE LA CALIDAD DEL SOFT,num_reprob_GESTION DE LA CALIDAD DEL SOFT,INGENIERIA DE REQUISITOS,num_reprob_INGENIERIA DE REQUISITOS,ARQUITECTURA DE SOFTWARE,num_reprob_ARQUITECTURA DE SOFTWARE,ARQUITECTURA EMPRESARIAL,num_reprob_ARQUITECTURA EMPRESARIAL,MODELADO DEL USUARIO,num_reprob_MODELADO DEL USUARIO,PRACTICUM III SERVICIO COMUNIT,num_reprob_PRACTICUM III SERVICIO COMUNIT,DESARROLLO BASADO EN PLAT WEB,num_reprob_DESARROLLO BASADO EN PLAT WEB,DESARROLLO BASADO EN PLATAF MO,num_reprob_DESARROLLO BASADO EN PLATAF MO,SISTEMAS DISTRIBUIDOS,num_reprob_SISTEMAS DISTRIBUIDOS,APLICACION DE INTERA HUMA COMP,num_reprob_APLICACION DE INTERA HUMA COMP,APLICACION DE MATEMATICAS Y ES,num_reprob_APLICACION DE MATEMATICAS Y ES,ORGANIZACION Y ADMIN DE INFRAE,num_reprob_ORGANIZACION Y ADMIN DE INFRAE,FUNDAMENTOS Y APLICACION DE SE,num_reprob_FUNDAMENTOS Y APLICACION DE SE,PRACTICUM 4.1,num_reprob_PRACTICUM 4.1,PROGRAMACION INTEGRATIVA,num_reprob_PROGRAMACION INTEGRATIVA,GOBERNANZA DE TECNOL DE INFOR,num_reprob_GOBERNANZA DE TECNOL DE INFOR,PROYECTOS DE TECNOLO DE INFORM,num_reprob_PROYECTOS DE TECNOLO DE INFORM,PRACTICUM 4.2,num_reprob_PRACTICUM 4.2,EVALUACION DE LA SEGUR EN SIST,num_reprob_EVALUACION DE LA SEGUR EN SIST,COMPOSICION DE TEXTOS CIENTIFI,num_reprob_COMPOSICION DE TEXTOS CIENTIFI,EMPRENDIMIENTO,num_reprob_EMPRENDIMIENTO
0,MCNO16001921933764,8.8,0,9.5,0,NaN,NaN,NaN,NaN,9.2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0,9.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.4,0,NaN,NaN,9.2,0,NaN,NaN,NaN,NaN,NaN,NaN,8.85,0,9.4,0,NaN,NaN,NaN,NaN,NaN,0,8.49,0,9.08,0,8.98,0,9.52,0,NaN,NaN,9.49,0,NaN,NaN,9.55,0,9.0,0,8.84,0,8.44,0,NaN,NaN,9.05,0,9.29,0,9.3,0,9.58,0,NaN,NaN,9.63,0,9.03,0,8.85,0,9.06,0,8.84,0,9.54,0


### Obtener datos para el caso 1

In [ ]:
def create_progress_bar(total, description):
    return tqdm(
        total = total, 
        desc = description, 
        position = 0 
    )

def close_progress_bar(progress_bar):
    progress_bar.clear()
    progress_bar.close()

def get_data_by_subjects(columns_grades, fake_identification, student_subjects):  
    student_grades_data = pd.DataFrame(columns=columns_grades) 
    student_grades_data = pd.concat([student_grades_data, pd.DataFrame({'fake_identificacion': [fake_identification]})])
    for subject_name in student_subjects:
        subject_final_score, subject_total_failures  = get_grades_data_by_recent_period_and_subject(fake_identification, subject_name)
        student_grades_data[subject_name] = subject_final_score
        student_grades_data[f'num_reprob_{subject_name}'] = subject_total_failures

    return student_grades_data

In [ ]:
columns_grades = set_grades_columns()
fake_identifications = grades_data.fake_identificacion.unique()
progress_bar = create_progress_bar(len(fake_identifications), 'Estudiantes procesados')
students_processed = 1
integrated_data_case1 = pd.DataFrame()

for fake_identification in fake_identifications:
    # Obtener datos de matriculas
    student_enrollment_data = get_enrollment_data_by_recent_period(fake_identification)

    # Obtener datos de notas
    student_subjects = get_student_subjects(fake_identification)
    student_grades_data = get_data_by_subjects(columns_grades, fake_identification, student_subjects)
    
    # Integrar datos de matriculas con datos de notas
    student_integrated_data = pd.merge(student_enrollment_data, student_grades_data, on='fake_identificacion')
    integrated_data_case1 = pd.concat([integrated_data_case1, student_integrated_data])
    
    # Actualizar progressbar
    students_processed =+ 1
    progress_bar.update(students_processed)

integrated_data_case1.columns = integrated_data_case1.columns.str.lower().str.replace(' ', '_')
integrated_data_case1.to_csv('integrated-data-case01.csv', index=False)
close_progress_bar(progress_bar)

Estudiantes procesados: 100%|██████████| 1938/1938 [04:00<00:00,  8.07it/s]


In [ ]:
integrated_data_case1

,fake_identificacion,ciclo_academico,codigo_programa,codigo_malla,codigo_periodo,region,zona,centro,tipo_de_est_programa,tipo_de_ingreso,periodo_admision,estatus_cursos,estatus_de_pago,descripcion_de_becas,porcentaje_de_beca,descripcion_descuento,porcentaje_des_matricula,monto_descuento_automatico,convenios_y_contratos,formas_de_pago,tipo_de_pago,provincia,canton,edad,sexo,discapacidad,tipo_discapacidad,porcentaje_de_discapacidades,numero_discapacidades,comunicacion_de_datos,num_reprob_comunicacion_de_datos,fundamentos_de_hardware,num_reprob_fundamentos_de_hardware,estructuras_discretas,num_reprob_estructuras_discretas,estructuras_de_datos,num_reprob_estructuras_de_datos,comunicacion_tecnica_y_profesi,num_reprob_comunicacion_tecnica_y_profesi,fundamentos_de_interaccion_hum,num_reprob_fundamentos_de_interaccion_hum,metodologia_de_la_invest_y_tec,num_reprob_metodologia_de_la_invest_y_tec,computacion_y_sociedad,num_reprob_computacion_y_sociedad,fundamentos_de_tecnologias,num_reprob_fundamentos_de_tecnologias,arquitectura_de_computadoras,num_reprob_arquitectura_de_computadoras,modelado_de_sistemas,num_reprob_modelado_de_sistemas,practicum_ii,num_reprob_practicum_ii,estadistica_para_las_ing_y_arq,num_reprob_estadistica_para_las_ing_y_arq,calculo_diferencial,num_reprob_calculo_diferencial,fundamentos_de_ingenieria_de_s,num_reprob_fundamentos_de_ingenieria_de_s,fundamentos_de_base_de_datos,num_reprob_fundamentos_de_base_de_datos,administracion_de_base_de_dato,num_reprob_administracion_de_base_de_dato,algoritmos_y_resolucion_de_pro,num_reprob_algoritmos_y_resolucion_de_pro,computacion_ubicua,num_reprob_computacion_ubicua,fundamentos_de_programacion,num_reprob_fundamentos_de_programacion,programacion_orientada_a_objet,num_reprob_programacion_orientada_a_objet,tecnologias_web,num_reprob_tecnologias_web,desarrollo_web,num_reprob_desarrollo_web,etica_y_moral,num_reprob_etica_y_moral,"humanismo,_universidad_y_cultu","num_reprob_humanismo,_universidad_y_cultu",antropologia_basica,num_reprob_antropologia_basica,fundamentos_matematicos,num_reprob_fundamentos_matematicos,algebra_lineal,num_reprob_algebra_lineal,calculo_integral,num_reprob_calculo_integral,redes_de_dispositivos,num_reprob_redes_de_dispositivos,arquitectura_de_redes,num_reprob_arquitectura_de_redes,plataformas_emergentes,num_reprob_plataformas_emergentes,gestion_de_proyectos,num_reprob_gestion_de_proyectos,practicum_i,num_reprob_practicum_i,planificacion_estrategica_y_si,num_reprob_planificacion_estrategica_y_si,metodologias_de_desarrollo,num_reprob_metodologias_de_desarrollo,gestion_de_la_calidad_del_soft,num_reprob_gestion_de_la_calidad_del_soft,ingenieria_de_requisitos,num_reprob_ingenieria_de_requisitos,arquitectura_de_software,num_reprob_arquitectura_de_software,arquitectura_empresarial,num_reprob_arquitectura_empresarial,modelado_del_usuario,num_reprob_modelado_del_usuario,practicum_iii_servicio_comunit,num_reprob_practicum_iii_servicio_comunit,desarrollo_basado_en_plat_web,num_reprob_desarrollo_basado_en_plat_web,desarrollo_basado_en_plataf_mo,num_reprob_desarrollo_basado_en_plataf_mo,sistemas_distribuidos,num_reprob_sistemas_distribuidos,aplicacion_de_intera_huma_comp,num_reprob_aplicacion_de_intera_huma_comp,aplicacion_de_matematicas_y_es,num_reprob_aplicacion_de_matematicas_y_es,organizacion_y_admin_de_infrae,num_reprob_organizacion_y_admin_de_infrae,fundamentos_y_aplicacion_de_se,num_reprob_fundamentos_y_aplicacion_de_se,practicum_4.1,num_reprob_practicum_4.1,programacion_integrativa,num_reprob_programacion_integrativa,gobernanza_de_tecnol_de_infor,num_reprob_gobernanza_de_tecnol_de_infor,proyectos_de_tecnolo_de_inform,num_reprob_proyectos_de_tecnolo_de_inform,practicum_4.2,num_reprob_practicum_4.2,evaluacion_de_la_segur_en_sist,num_reprob_evaluacion_de_la_segur_en_sist,composicion_de_textos_cientifi,num_reprob_composicion_de_textos_cientifi,emprendimiento,num_reprob_emprendimiento
0,SLXA73875862178452,4,INFO_D1,IFD6,202264,GUAYAQUIL,GRAN-GYE,GUAYAQUIL,CONTINUO,ADAPTA